# 1 Purpose

### 1.0.1 Imports

In [2]:
import pandas as pd
import numpy as np
pd.set_option('mode.chained_assignment', None)

## 1.1 Data Wrangling

In [9]:
drop = 'Unnamed: 0'

seasons = pd.read_csv('../data/processed/players_final.csv')

caps = pd.read_csv('../data/processed/salary_cap_history.csv').drop(drop, axis = 1)
caps = caps.rename(columns = {'year':'Year'})

teams = pd.read_csv('../data/processed/teams_final.csv').drop(drop, axis = 1)
teams = teams[['Tm','Year','W','L','W/L%','Rank']]

In [11]:
seasons_with_caps = pd.merge(seasons, caps, on = 'Year', how = 'left')
full = pd.merge(seasons_with_caps, teams, on = ['Tm','Year'])
full.columns

Index(['Full Name', 'Pos', 'cluster', 'index', 'Last Name', 'First Name',
       'Year', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA',
       '3P%', '2P', '2PA', '2P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB',
       'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'ORtg', 'DRtg', 'PER', 'TS%',
       '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%',
       'USG%', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP',
       'salary', 'Dist.', '2P_A', '0-3_A', '3-10_A', '10-16_A', '16-3pt_A',
       '3P_A', '0-3%', '3-10%', '10-16%', '16-3pt%', 'Astd_2P', 'Dunk%',
       'Dunks_Md', 'Astd_3P', '%Corner3_A', 'Corner_3P%', 'Heaves_A',
       'Heaves_Md', 'salary_cap', 'W', 'L', 'W/L%', 'Rank'],
      dtype='object')

# 2 def assessment():

In [18]:
def assessment(year = 'any', cluster = 'any', stat = 'WS', highseed = False):
    """Takes year, cluster, and stat as inputs and outputs a boolean for whether a player is overpaid. 
    Signifying highseed as True sets the bar higher for establishing value."""
    if year == 'any':
        df = full
    else:    
        df = full[full.Year == year].loc[:,['Full Name','Tm','Year','cluster',
                                            'salary','salary_cap','G','GS',
                                            'MP','PER','VORP','WS','USG%','W',
                                            'L','BPM']]
    df['burden'] = round(df['salary']/df['salary_cap'],3)
    
    if year != 'any':
        df = df[['Full Name','Tm','W','L','salary','salary_cap','burden','cluster', stat]]
    else:
        df = df[['Full Name','Year','Tm','W','L','salary','salary_cap','burden','cluster', stat]]
        df.Year = df.Year.astype('int')
        
    if cluster != 'any':
        cluster_avg = df[df.cluster == cluster].groupby('cluster').mean().salary[0]
        df['cluster_avg'] = cluster_avg
        df = df[df.cluster == cluster]
        df = df.drop(columns = ['cluster','cluster_avg'], axis = 1)    
    
    if highseed == False:
        wincount = 44
    else:
        wincount = 49
    
    if stat == 'WS':
        df['exp_WS'] = round(df.burden * wincount,1)
        df['overpaid'] = df.exp_WS > df.WS
    elif stat == 'BPM': ########################### this calculation needs work #############################
        df['exp_BPM'] = round(df.burden * ((wincount * 29.03) - 1190.63),1)
        df['overpaid'] = df.exp_BPM > df.BPM
    
    
    compensation = df.sort_values('burden', ascending = False)
    
    if year != 'any':
        print('Salary cap for '+str(year)+':\t',str(compensation.iloc[0,5]))
        compensation = compensation.drop(columns = 'salary_cap')
        
    if cluster != 'any':
        print(cluster + ' average salary: ' + str(int(cluster_avg)))
    
    return compensation

In [19]:
# assessment(year = 2018)

Salary cap for 2018:	 101869000


,Full Name,Tm,W,L,salary,burden,cluster,WS,exp_WS,overpaid
1241,Stephen Curry,GSW,57,25,37457154.0,0.368,Combo Guard,9.7,16.2,True
376,LeBron James,LAL,37,45,35654150.0,0.350,Combo Guard,7.2,15.4,True
897,Chris Paul,HOU,53,29,35654150.0,0.350,Floor General,6.6,15.4,True
4,Russell Westbrook,OKC,49,33,35665000.0,0.350,Floor General,6.8,15.4,True
1835,Kyle Lowry,TOR,58,24,32700000.0,0.321,Floor General,6.6,14.1,True
762,Blake Griffin,DET,41,41,31873932.0,0.313,Slasher Shot Creator,8.0,13.8,True
896,James Harden,HOU,53,29,30570000.0,0.300,Combo Guard,15.2,13.2,False
163,Mike Conley,MEM,33,49,30521115.0,0.300,Floor General,8.0,13.2,True
2,Paul George,OKC,49,33,30560700.0,0.300,Lockdown,11.9,13.2,True
1242,Kevin Durant,GSW,57,25,30000000.0,0.294,Slasher Shot Creator,11.5,12.9,True
